# HANA Cloud - Predicitive Analysis Library Hands On


### Documentation
- SAP HANA Python Client API for Machine Learning Algorithms: https://pypi.org/project/hana-ml/

- SAP HANA Predictive Analysis Library (PAL): https://help.sap.com/viewer/2cfbc5cf2bc14f028cfbe2a2bba60a50/1.0.12/en-US

SAP HANA ML Library
You will be using the 'SAP HANA Python Client API for Machine Learning Algorithm'.

### 1.	How can we directly access the data in our HANA? 

In [ ]:
!pip install hana_ml

In [1]:
import hana_ml

In [ ]:
hana_address = #your hostname as string
hana_port = #your port as integer
hana_user = #your user as string
hana_password = #your password as string
hana_encrypt = 'true' #for HANA Cloud

In [ ]:
import hana_ml.dataframe as dataframe

# Establish connection
conn = dataframe.ConnectionContext(address = hana_address,
                                   port = hana_port, 
                                   user = hana_user, 
                                   password = hana_password, 
                                   encrypt = hana_encrypt,
                                   sslValidateCertificate = 'false')

Through a HANA Key we are able to hide our login credentials.

In [ ]:
import hana_ml.dataframe as dataframe

# Establish connection
conn = dataframe.ConnectionContext(userkey = 'MYHANACLOUD',
                                   encrypt = 'true',
                                   sslValidateCertificate = 'false')

In [4]:
import pandas as pd
#load data Predictive_Quality_Use_Case.csv, change path to your directory
df = pd.read_csv(r"./pred_quality.csv")
df.head()

,supplier,machine,quality,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6,sensor7,sensor8,sensor9,sensor10
0,SupE,MachA,0,0.341645,0.502370,0.431381,0.201660,0.521939,0.436171,0.281260,0.478072,0.378124,0.458864
1,SupE,MachA,0,0.298768,0.189764,0.359475,0.246197,0.427249,0.222523,0.522413,0.210466,0.519614,0.277754
2,SupF,MachC,0,0.441123,0.373059,0.434842,0.359969,0.453749,0.540650,0.495720,0.478441,0.299070,0.208134
3,SupF,MachB,0,0.307060,0.388752,0.490346,0.286483,0.289103,0.580277,0.348547,0.475260,0.233768,0.152235
4,SupC,MachA,0,0.378825,0.132827,0.163909,0.341255,0.360088,0.377897,0.370030,0.389969,0.399875,0.348875


In [5]:
#change columns to upper string
df.columns = map(str.upper, df.columns)

In [6]:
#insert a product ID, which will later be used as key
df.insert(0, 'PRODUCT_ID', df.reset_index().index)

In [7]:
#control a sample of the data
df.head()

,PRODUCT_ID,SUPPLIER,MACHINE,QUALITY,SENSOR1,SENSOR2,SENSOR3,SENSOR4,SENSOR5,SENSOR6,SENSOR7,SENSOR8,SENSOR9,SENSOR10
0,0,SupE,MachA,0,0.341645,0.502370,0.431381,0.201660,0.521939,0.436171,0.281260,0.478072,0.378124,0.458864
1,1,SupE,MachA,0,0.298768,0.189764,0.359475,0.246197,0.427249,0.222523,0.522413,0.210466,0.519614,0.277754
2,2,SupF,MachC,0,0.441123,0.373059,0.434842,0.359969,0.453749,0.540650,0.495720,0.478441,0.299070,0.208134
3,3,SupF,MachB,0,0.307060,0.388752,0.490346,0.286483,0.289103,0.580277,0.348547,0.475260,0.233768,0.152235
4,4,SupC,MachA,0,0.378825,0.132827,0.163909,0.341255,0.360088,0.377897,0.370030,0.389969,0.399875,0.348875


In [8]:
#Create a SAP HANA dataframe and point it to the table with the uploaded data.
df_remote = dataframe.create_dataframe_from_pandas(connection_context = conn, 
                                                   pandas_df = df, 
                                                   table_name = 'PREDICTIVEQUALITY',
                                                   force = True,
                                                   replace = False)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:08<00:00,  2.87s/it]


### 2.	How can we explore our data and react to data quality issues early? 

In [9]:
#control the size of the data
df_remote.count()

125000

In [10]:
#control the variable types in SAP HANA
df_remote.dtypes()

[('PRODUCT_ID', 'INT', 10, 10, 10, 0),
 ('SUPPLIER', 'NVARCHAR', 5000, 5000, 5000, 0),
 ('MACHINE', 'NVARCHAR', 5000, 5000, 5000, 0),
 ('QUALITY', 'INT', 10, 10, 10, 0),
 ('SENSOR1', 'DOUBLE', 15, 15, 15, 0),
 ('SENSOR2', 'DOUBLE', 15, 15, 15, 0),
 ('SENSOR3', 'DOUBLE', 15, 15, 15, 0),
 ('SENSOR4', 'DOUBLE', 15, 15, 15, 0),
 ('SENSOR5', 'DOUBLE', 15, 15, 15, 0),
 ('SENSOR6', 'DOUBLE', 15, 15, 15, 0),
 ('SENSOR7', 'DOUBLE', 15, 15, 15, 0),
 ('SENSOR8', 'DOUBLE', 15, 15, 15, 0),
 ('SENSOR9', 'DOUBLE', 15, 15, 15, 0),
 ('SENSOR10', 'DOUBLE', 15, 15, 15, 0)]

The variable QUALITY is binary and labels all products of bad quality with a 1. Since this is a categorical variable we transform it to type NVARCHAR with the following command. 

In [11]:
#transform the variable QUALITY
df_remote = df_remote.cast('QUALITY', 'NVARCHAR(20)')

In [20]:
df_remote.agg([('count', 'PRODUCT_ID', 'N')], group_by='QUALITY').collect()

,QUALITY,N
0,0,100000
1,1,25000


In [12]:
#control the variable types
df_remote.dtypes()

[('PRODUCT_ID', 'INT', 10, 10, 10, 0),
 ('SUPPLIER', 'NVARCHAR', 5000, 5000, 5000, 0),
 ('MACHINE', 'NVARCHAR', 5000, 5000, 5000, 0),
 ('QUALITY', 'NVARCHAR', 20, 20, 20, 0),
 ('SENSOR1', 'DOUBLE', 15, 15, 15, 0),
 ('SENSOR2', 'DOUBLE', 15, 15, 15, 0),
 ('SENSOR3', 'DOUBLE', 15, 15, 15, 0),
 ('SENSOR4', 'DOUBLE', 15, 15, 15, 0),
 ('SENSOR5', 'DOUBLE', 15, 15, 15, 0),
 ('SENSOR6', 'DOUBLE', 15, 15, 15, 0),
 ('SENSOR7', 'DOUBLE', 15, 15, 15, 0),
 ('SENSOR8', 'DOUBLE', 15, 15, 15, 0),
 ('SENSOR9', 'DOUBLE', 15, 15, 15, 0),
 ('SENSOR10', 'DOUBLE', 15, 15, 15, 0)]

In [25]:
#describe the data in SAP HANA
df_remote.describe().collect()

,column,count,unique,nulls,mean,std,min,max,median,25_percent_cont,25_percent_disc,50_percent_cont,50_percent_disc,75_percent_cont,75_percent_disc
0,PRODUCT_ID,125000,125000,0,62499.500000,36084.536161,0.0,124999.0,62500.000000,31249.750000,31249.000000,62499.500000,62499.000000,93749.250000,93749.000000
1,SENSOR1,125000,124982,0,0.479726,0.153780,0.0,1.0,0.478589,0.354348,0.354348,0.478589,0.478587,0.606131,0.606128
2,SENSOR2,125000,124991,0,0.485109,0.205528,0.0,1.0,0.485640,0.309291,0.309281,0.485640,0.485637,0.660287,0.660286
3,SENSOR3,125000,124988,0,0.497013,0.208116,0.0,1.0,0.496390,0.318205,0.318204,0.496390,0.496385,0.675093,0.675093
4,SENSOR4,125000,124983,0,0.459679,0.149585,0.0,1.0,0.466757,0.337220,0.337219,0.466757,0.466756,0.579259,0.579259
5,SENSOR5,125000,124977,0,0.509524,0.156953,0.0,1.0,0.510624,0.379915,0.379915,0.510624,0.510622,0.638532,0.638532
6,SENSOR6,125000,124982,0,0.509260,0.158821,0.0,1.0,0.509265,0.377351,0.377350,0.509265,0.509264,0.641560,0.641557
7,SENSOR7,125000,124986,0,0.483033,0.159582,0.0,1.0,0.482779,0.349749,0.349749,0.482779,0.482776,0.616109,0.616108
8,SENSOR8,125000,124985,0,0.485639,0.158661,0.0,1.0,0.485044,0.352291,0.352289,0.485044,0.485043,0.618902,0.618901
9,SENSOR9,125000,124987,0,0.469235,0.152392,0.0,1.0,0.468354,0.342879,0.342879,0.468354,0.468352,0.592790,0.592790


In [13]:
#create a Data Report for further exploration
from hana_ml.visualizers.dataset_report import DatasetReportBuilder
datasetReportBuilder = DatasetReportBuilder()
datasetReportBuilder.build(df_remote, key="PRODUCT_ID")

Generating dataset report...: 100%|███████████████████████████████████| 8/8 [00:35<00:00,  4.49s/it]


In [14]:
#generate Data Report directly in Jupyter Notebook
datasetReportBuilder.generate_notebook_iframe_report()

In order to review the dataset report better, you need to adjust the size of the left area or hide the left area temporarily!


### 3.	How can we leverage the computing power of our HANA in our machine learning use case?  

In [15]:
#create training and testing set
from hana_ml.algorithms.pal import partition
hdf_train, hdf_test, hdf_val = partition.train_test_val_split( random_seed = 1017,
                                                               data = df_remote, 
                                                               training_percentage = 0.8, 
                                                                testing_percentage = 0.1,
                                                                validation_percentage = 0.1)

In [16]:
#control the size of the training and testing set
print('Size of training subset: ' + str(hdf_train.count()))
print('Size of test subset: ' + str(hdf_test.count()))

Size of training subset: 100000
Size of test subset: 12500


In [17]:
# Union train and validation data into one set
df_trainval=hdf_train.select('*', ('1', 'TRAIN_VAL_INDICATOR' )).union(hdf_val.select('*', ('2', 'TRAIN_VAL_INDICATOR' )))

display(df_trainval.head(5).collect())

,PRODUCT_ID,SUPPLIER,MACHINE,QUALITY,SENSOR1,SENSOR2,SENSOR3,SENSOR4,SENSOR5,SENSOR6,SENSOR7,SENSOR8,SENSOR9,SENSOR10,TRAIN_VAL_INDICATOR
0,22,SupC,MachC,0,0.598684,0.544622,0.653084,0.467637,0.833161,0.600017,0.580115,0.558271,0.494238,0.806028,1
1,57,SupC,MachC,0,0.336701,0.442155,0.373946,0.341537,0.380335,0.471353,0.475528,0.380412,0.160266,0.293423,1
2,71,SupC,MachC,0,0.586596,0.580597,0.643189,0.532847,0.576641,0.742735,0.625323,0.677316,0.642405,0.565801,1
3,78,SupC,MachC,0,0.302299,0.518902,0.351178,0.405238,0.296945,0.325774,0.370321,0.456620,0.384333,0.358422,1
4,87,SupC,MachC,0,0.589654,0.676642,0.772341,0.623043,0.642281,0.645835,0.667018,0.578739,0.597443,0.486433,1


Let us now train or random forest on the training set. First, we set the numbers of trees very high, to see where the Out of Bag error converges. After optimizing the numbers of trees we will take a closer look at the variables considered at each split. 

In [39]:
# Train the Station classifer model using PAL HybridGradientBoostingTree
from hana_ml.algorithms.pal.unified_classification import UnifiedClassification

# Initialize the model object 
hgbc = UnifiedClassification(func='HybridGradientBoostingTree',
                            n_estimators = 101, max_depth=5,
                            split_method='histogram', max_bin_num=1000, 
                            #feature_grouping=True, tol_rate=0.001,
                            learning_rate=0.1, split_threshold=0.1,
                            resampling_method='stratified_cv', fold_num=5, evaluation_metric = 'error_rate', ref_metric=['auc']
                            )

# Execute the training of the model
hgbc.fit(data=df_trainval, 
         key='PRODUCT_ID', 
         features=['SUPPLIER','MACHINE','SENSOR1','SENSOR2','SENSOR3','SENSOR4','SENSOR5','SENSOR6','SENSOR7','SENSOR8','SENSOR9','SENSOR10'],
         label='QUALITY', categorical_variable= ['SUPPLIER', 'MACHINE', 'QUALITY'],
         ntiles=20,  build_report=True,
         partition_method='user_defined', purpose='TRAIN_VAL_INDICATOR')

display(hgbc.runtime)

14.351041316986084

In [36]:
# Build Model Report
from hana_ml.visualizers.unified_report import UnifiedReport
UnifiedReport(hgbc).build().display()

In order to review the unified classification model report better, you need to adjust the size of the left area or hide the left area temporarily!


In [40]:
# Test model generalization using the test data-subset, not used during training
scorepredictions, scorestats, scorecm, scoremetrics = hgbc.score(data=hdf_test , key= 'PRODUCT_ID', label='QUALITY', 
                                                                 ntiles=20, 
                                                                  thread_ratio=1.0)
display(hgbc.runtime)
display(scorestats.sort('CLASS_NAME').collect())
display(scorecm.filter('COUNT != 0').collect())
#display(scoremetrics.collect())

38.16511154174805

,STAT_NAME,STAT_VALUE,CLASS_NAME
0,AUC,0.9684766144,None
1,ACCURACY,0.90616,None
2,KAPPA,0.6877432035884072,None
3,MCC,0.6938425680131939,None
4,RECALL,0.966054032339058,0
5,PRECISION,0.9201262674574325,0
6,F1_SCORE,0.9425309881926411,0
7,SUPPORT,9957,0
8,RECALL,0.6716476602438065,1
9,PRECISION,0.8347996089931574,1


,ACTUAL_CLASS,PREDICTED_CLASS,COUNT
0,0,0,9619
1,0,1,338
2,1,0,835
3,1,1,1708


### 4.	How can we save and create different versions of our results

In [ ]:
#create Model storage
from hana_ml.model_storage import ModelStorage 
MODEL_SCHEMA = 'YANNICK' # HANA schema in which models are to be saved 
model_storage = ModelStorage(connection_context=conn, schema = MODEL_SCHEMA) 
rf.name = 'Unified Classification HGBT Model' 
model_storage.save_model(model=rf, if_exists = 'replace')

In [ ]:
# Lists model
list_models = model_storage.list_models()
print(list_models)